# Logit

In [74]:
using DiscreteChoice

In [75]:
using DataFrames, ForwardDiff

In [76]:
df = readtable("australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b

head(df)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
1,4,1,0,0,0,0,1,0,0,0,0,1,0,35,0,0,0,69,34,35,0,70,71,70,30
2,4,1,0,0,0,0,1,0,0,0,0,1,0,30,0,0,0,64,44,53,0,68,84,85,50
3,4,1,0,0,0,0,1,0,0,0,0,1,0,40,0,0,0,69,34,35,0,129,195,149,101
4,4,1,0,0,0,0,1,0,0,0,0,1,0,70,0,0,0,64,44,53,0,59,79,81,32
5,4,1,0,0,0,0,1,0,0,0,0,1,0,45,0,0,0,64,44,53,0,82,93,94,99
6,2,1,0,0,0,0,1,0,0,0,0,1,0,20,0,0,0,69,40,35,0,70,57,58,43


### For each individual, the logit is

$$L_{ij}(\beta) = P_{ij}(\beta) = \frac{e^{\mu V_{ij}(\beta)}}{\sum_{m=1}^{|A(i)|} e^{\mu V_{im}(\beta)}}$$

In [77]:
function individual(β::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    data = convert(Array, df[i, 2:n_parameters])
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function logit(β::Vector)
        t = 0.0
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative) - c)
        end
        return 1 / (1 + t)
    end
    
    return logit
end

individual (generic function with 1 method)

### So the function we have to optimize is
    
$$\max_{\beta} LL(\beta) = \max_{\beta} \frac{1}{I} {\sum_{i=1}^{I} \ln P_{ij}(\beta)}$$

In [78]:
function f(β::Vector)
    model = 0.0
    i = 1
    while i <= n_individuals
        logit = individual(β, i)
        model += log(logit(β))
        i += 1
    end
    return -model/n_individuals
end

f (generic function with 1 method)

In [79]:
g(x::Vector) = ForwardDiff.gradient(f, x);

In [80]:
H(x::Vector) = ForwardDiff.hessian(f, x);

### Using the trust-region method with the truncated conjugate gradient:

In [81]:
optimize(f, g, H, tcg, zeros(6))

([5.20744, 3.86904, 3.16319, 0.013287, -0.0961248, -0.0155015], 12)

### What if we would want to use an Hessian approximation?

In [82]:
optimize(f, g, BFGS!, tcg, zeros(6), true)

([5.20744, 3.86904, 3.16319, 0.013287, -0.0961248, -0.0155015], 46)